In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import Normalizer, MinMaxScaler, LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras import metrics
from sklearn.utils import shuffle

####atom://teletype/portal/3f155216-29d7-4279-882f-d6aec0f4c85b

df_train = pd.read_csv('/content/drive/My Drive/COVID_19/train.csv')
df_test = pd.read_csv('/content/drive/My Drive/COVID_19/test.csv')

train_data = df_train.drop(
    ['Id', 'County', 'Province_State', 'Country_Region'], axis=1)
#test_data = df_test.drop(
#    ['County', 'Province_State', 'Country_Region'], axis=1)
train_data.set_index('Date', inplace=True)
#test_data.set_index('Date', inplace=True)

train_confirm = train_data[train_data['Target'] == 'ConfirmedCases']
train_confirm = train_confirm.drop(['Target'], axis = 1)
train_confirm['TargetValue'] = np.where(train_confirm['TargetValue'] <=0, 0, train_confirm['TargetValue'])
#print(train_confirm)

X = train_confirm.iloc[:, 0: 4].to_numpy()
Y = train_data.iloc[:, 4: 5].to_numpy()

# MinMaxScaling

sc_pop = MinMaxScaler(feature_range=(0, 1))
sc_tg = MinMaxScaler(feature_range=(0, 1))
X[:, 0:1] = sc_pop.fit_transform(X[:, 0:1])
X[:, 2:3] = sc_tg.fit_transform(X[:, 2:3])


X = X.reshape(-1,104,3)
print(X.shape)
#print(df_train.dtypes)
#print(X)

def multivariate_data(dataset, target, start_index, end_index, time_step) :
	data=list()
	label =list()

	start_index = start_index + time_step
	for i in range(start_index, end_index) :
		indices = range(i-time_step, i)
		data.append(dataset[indices])
		label.append(target[i])

	return np.array(data), np.array(label)

time_step = 40
partition = 104-4-time_step
X_train, Y_train = multivariate_data(X[0,:,:], X[0,:,2], 0, 104, time_step)

for i in range(1,3463) :
	X_dummy, Y_dummy = multivariate_data(X[i,:,:], X[i,:,2], 0, 104, time_step)
	X_train = np.concatenate((X_train, X_dummy), axis = 0)
	Y_train = np.concatenate((Y_train, Y_dummy), axis = 0)

print(X_train.shape)
print(Y_train.shape)


(3463, 104, 3)
(221632, 40, 3)
(221632,)


In [7]:
regressor = Sequential()

regressor.add(LSTM(units = 16, return_sequences = True, input_shape = (X_train.shape[1], 3)))
regressor.add(Dropout(0.3))
regressor.add(LSTM(units = 16, return_sequences = True))
regressor.add(Dropout(0.3))
regressor.add(LSTM(units = 16, return_sequences = True))
regressor.add(Dropout(0.3))
regressor.add(LSTM(units = 16))
regressor.add(Dropout(0.3))
regressor.add(Dense(units = 1))
regressor.compile(optimizer = 'RMSprop', loss = 'mean_squared_error')
regressor.fit(X_train, Y_train, epochs = 7, batch_size = 32)


Epoch 1/7
6926/6926 [==============================] - 66s 10ms/step - loss: 7.9666e-05
Epoch 2/7
6926/6926 [==============================] - 66s 10ms/step - loss: 8.1239e-05
Epoch 3/7
6926/6926 [==============================] - 67s 10ms/step - loss: 6.8386e-05
Epoch 4/7
6926/6926 [==============================] - 66s 10ms/step - loss: 6.9338e-05
Epoch 5/7
6926/6926 [==============================] - 66s 10ms/step - loss: 5.1502e-05
Epoch 6/7
6926/6926 [==============================] - 66s 10ms/step - loss: 4.7582e-05
Epoch 7/7
6926/6926 [==============================] - 66s 10ms/step - loss: 3.6580e-05


In [8]:
test_data = df_test.drop(
    ['ForecastId','County', 'Province_State', 'Country_Region'], axis=1)
test_data.set_index('Date', inplace=True)
test_confirm = test_data[test_data['Target'] == 'ConfirmedCases']
test_confirm = test_confirm.drop(['Target'], axis = 1)

x_test = test_confirm.iloc[:, 0: 4].values
(a,b) = x_test.shape
x_test[:, 0:1] = sc_pop.fit_transform(x_test[:, 0:1])

X_modify = np.zeros(shape = (a,b+1))
X_modify[:,:-1] = x_test
X_modify = X_modify.reshape(-1,45,3)

X_t1 = X_modify[0,:,:]
X_t2 = X[0,:,:]
X_t3 = np.concatenate((X_t2,X_t1), axis = 0)
st_index = 104 - time_step
X_t3 = X_t3[st_index:149,:]
X_test1, Y_d1 = multivariate_data(X_t3, X_t3[:,2], 0, time_step + 45, time_step)
print(X_test1.shape)

for i in range (1,3463) :
	x_t1 = X_modify[i,:,:]
	X_t2 = X[i,:,:]
	X_t3 = np.concatenate((X_t2,X_t1), axis = 0)
	X_t3 = X_t3[st_index:149,:]
	X_test_dummy, Y_d1 = multivariate_data(X_t3, X_t3[:,2], 0, time_step + 45, time_step)
	X_test1 = np.concatenate((X_test1, X_test_dummy), axis =0)

predicted_test = regressor.predict(X_test1)
predicted_test = sc_tg.inverse_transform(predicted_test)
print(predicted_test)

(45, 40, 3)
[[116.74444 ]
 [131.80185 ]
 [140.70868 ]
 ...
 [ -3.100041]
 [ -3.100041]
 [ -3.100041]]


In [9]:
pred_test_flat = predicted_test.flatten()
print(pred_test_flat)
pred_test_flat = pred_test_flat.astype(int)
print(pred_test_flat)

[116.74444  131.80185  140.70868  ...  -3.100041  -3.100041  -3.100041]
[116 131 140 ...  -3  -3  -3]


In [10]:
pred_test_flat.shape

(155835,)

In [25]:
df = pd.DataFrame(pred_test_flat)
my_list = [*range(2,935011,6)]
df['Id_1'] = my_list
df.rename(columns = {0 : 'Predicted_Results'}, inplace = True)
df['Predicted_Results'] = np.where(df['Predicted_Results'] <0, 0, df['Predicted_Results'])
df

,Predicted_Results,Id_1
0,116,2
1,131,8
2,140,14
3,139,20
4,126,26
...,...,...
155830,0,934982
155831,0,934988
155832,0,934994
155833,0,935000


In [27]:
s1 = df['Predicted_Results']
l1 = s1.tolist()
l2 = [i*2 for i in l1]
l1 = [i*0.95 for i in l2]
l2 = [round(i) for i in l1]
df2 = pd.DataFrame(l2)

my_list = [*range(3,935011,6)]
df2['Id_1'] = my_list
df2.rename(columns = {0 : 'Predicted_Results'}, inplace = True)

s1 = df['Predicted_Results']
l1 = s1.tolist()
l2 = [i*2 for i in l1]
l1 = [i*0.05 for i in l2]
l2 = [round(i) for i in l1]
df3 = pd.DataFrame(l2)

my_list = [*range(1,935011,6)]
df3['Id_1'] = my_list
df3.rename(columns = {0 : 'Predicted_Results'}, inplace = True)

result_confirmed_cases = pd.concat([df, df2, df3])
result_confirmed_cases.sort_values(by = ['Id_1'], inplace = True)
result_confirmed_cases

,Predicted_Results,Id_1
0,12,1
0,116,2
0,220,3
1,13,7
1,131,8
...,...,...
155833,0,935000
155833,0,935001
155834,0,935005
155834,0,935006


In [28]:
train_confirm1 = train_data[train_data['Target'] == 'Fatalities']
train_confirm1 = train_confirm1.drop(['Target'], axis = 1)
train_confirm1['TargetValue'] = np.where(train_confirm1['TargetValue'] <=0, 0, train_confirm1['TargetValue'])
#print(train_confirm)

X1 = train_confirm1.iloc[:, 0: 4].to_numpy()

# MinMaxScaling

X1[:, 0:1] = sc_pop.fit_transform(X1[:, 0:1])
X1[:, 2:3] = sc_tg.fit_transform(X1[:, 2:3])


X1 = X1.reshape(-1,104,3)
print(X1.shape)
#print(df_train.dtypes)
#print(X)

def multivariate_data(dataset, target, start_index, end_index, time_step) :
	data=list()
	label =list()

	start_index = start_index + time_step
	for i in range(start_index, end_index) :
		indices = range(i-time_step, i)
		data.append(dataset[indices])
		label.append(target[i])

	return np.array(data), np.array(label)

X_train1, Y_train1 = multivariate_data(X1[0,:,:], X1[0,:,2], 0, 104, time_step)

for i in range(1,3463) :
	X_dummy, Y_dummy = multivariate_data(X1[i,:,:], X1[i,:,2], 0, 104, time_step)
	X_train1 = np.concatenate((X_train1, X_dummy), axis = 0)
	Y_train1 = np.concatenate((Y_train1, Y_dummy), axis = 0)

print(X_train1.shape)
print(Y_train1.shape)


(3463, 104, 3)
(221632, 40, 3)
(221632,)


In [29]:
regressor1 = Sequential()

regressor1.add(LSTM(units = 16, return_sequences = True, input_shape = (X_train1.shape[1], 3)))
regressor1.add(Dropout(0.3))
regressor1.add(LSTM(units = 16, return_sequences = True))
regressor1.add(Dropout(0.3))
regressor1.add(LSTM(units = 16, return_sequences = True))
regressor1.add(Dropout(0.3))
regressor1.add(LSTM(units = 16))
regressor1.add(Dropout(0.3))
regressor1.add(Dense(units = 1))
regressor1.compile(optimizer = 'RMSprop', loss = 'mean_squared_error')
regressor1.fit(X_train1, Y_train1, epochs = 7, batch_size = 32)

Epoch 1/7
6926/6926 [==============================] - 67s 10ms/step - loss: 4.7147e-05
Epoch 2/7
6926/6926 [==============================] - 66s 10ms/step - loss: 2.9095e-05
Epoch 3/7
6926/6926 [==============================] - 66s 10ms/step - loss: 2.4805e-05
Epoch 4/7
6926/6926 [==============================] - 66s 10ms/step - loss: 2.5505e-05
Epoch 5/7
6926/6926 [==============================] - 66s 10ms/step - loss: 2.6499e-05
Epoch 6/7
6926/6926 [==============================] - 67s 10ms/step - loss: 2.1532e-05
Epoch 7/7
6926/6926 [==============================] - 66s 10ms/step - loss: 2.1328e-05


In [30]:
test_data = df_test.drop(
    ['ForecastId','County', 'Province_State', 'Country_Region'], axis=1)
test_data.set_index('Date', inplace=True)
test_confirm = test_data[test_data['Target'] == 'Fatalities']
test_confirm = test_confirm.drop(['Target'], axis = 1)

x_test = test_confirm.iloc[:, 0: 4].values
(a,b) = x_test.shape
x_test[:, 0:1] = sc_pop.fit_transform(x_test[:, 0:1])

X_modify = np.zeros(shape = (a,b+1))
X_modify[:,:-1] = x_test
X_modify = X_modify.reshape(-1,45,3)

X_t1 = X_modify[0,:,:]
X_t2 = X[0,:,:]
X_t3 = np.concatenate((X_t2,X_t1), axis = 0)
st_index = 104 - time_step
X_t3 = X_t3[st_index:149,:]
X_test1, Y_d1 = multivariate_data(X_t3, X_t3[:,2], 0, time_step + 45, time_step)

for i in range (1,3463) :
	x_t1 = X_modify[i,:,:]
	X_t2 = X[i,:,:]
	X_t3 = np.concatenate((X_t2,X_t1), axis = 0)
	X_t3 = X_t3[st_index:149,:]
	X_test_dummy, Y_d1 = multivariate_data(X_t3, X_t3[:,2], 0, time_step + 45, time_step)
	X_test1 = np.concatenate((X_test1, X_test_dummy), axis =0)

predicted_test = regressor1.predict(X_test1)
predicted_test = sc_tg.inverse_transform(predicted_test)
print(predicted_test)

(45, 40, 3)
[[ 9.166556 ]
 [11.171688 ]
 [13.773139 ]
 ...
 [ 2.8854387]
 [ 2.8854387]
 [ 2.8854387]]


In [31]:
pred_test_flat = predicted_test.flatten()
pred_test_flat = pred_test_flat.astype(int)
df4 = pd.DataFrame(pred_test_flat)
my_list = [*range(5,935011,6)]
df4['Id_1'] = my_list
df4.rename(columns = {0 : 'Predicted_Results'}, inplace = True)
df4['Predicted_Results'] = np.where(df4['Predicted_Results'] <0, 0, df4['Predicted_Results'])
df4

,Predicted_Results,Id_1
0,9,5
1,11,11
2,13,17
3,16,23
4,17,29
...,...,...
155830,2,934985
155831,2,934991
155832,2,934997
155833,2,935003


In [32]:
s1 = df4['Predicted_Results']
l1 = s1.tolist()
l2 = [i*2 for i in l1]
l1 = [i*0.95 for i in l2]
l2 = [round(i) for i in l1]
df5 = pd.DataFrame(l2)

my_list = [*range(6,935011,6)]
df5['Id_1'] = my_list
df5.rename(columns = {0 : 'Predicted_Results'}, inplace = True)

s1 = df4['Predicted_Results']
l1 = s1.tolist()
l2 = [i*2 for i in l1]
l1 = [i*0.05 for i in l2]
l2 = [round(i) for i in l1]
df6 = pd.DataFrame(l2)

my_list = [*range(4,935011,6)]
df6['Id_1'] = my_list
df6.rename(columns = {0 : 'Predicted_Results'}, inplace = True)

result_fatalities = pd.concat([df4, df5, df6])
result_fatalities.sort_values(by = ['Id_1'], inplace = True)
result_fatalities

,Predicted_Results,Id_1
0,1,4
0,9,5
0,17,6
1,1,10
1,11,11
...,...,...
155833,2,935003
155833,4,935004
155834,0,935008
155834,2,935009


In [34]:
result_total = pd.concat([result_confirmed_cases, result_fatalities])
result_total.sort_values(by = ['Id_1'], inplace = True)
result_total

,Predicted_Results,Id_1
0,12,1
0,116,2
0,220,3
0,1,4
0,9,5
...,...,...
155834,0,935006
155834,0,935007
155834,0,935008
155834,2,935009


In [41]:
df_submission = pd.read_csv('/content/drive/My Drive/COVID_19/submission.csv')
my_list = [*range(1,935011)]
df_submission['Id'] = my_list 
final_result = pd.merge(df_submission,result_total, left_on = 'Id', right_on ='Id_1', how = 'inner')
final_result.drop(['TargetValue','Id','Id_1','ForecastId'], axis =1, inplace = True)
final_result.rename({'Predicted_Results' : 'TargetValue'}, axis = 1, inplace = True)
final_result

,ForecastId_Quantile,TargetValue
0,1_0.05,12
1,1_0.5,116
2,1_0.95,220
3,2_0.05,1
4,2_0.5,9
...,...,...
935005,311669_0.5,0
935006,311669_0.95,0
935007,311670_0.05,0
935008,311670_0.5,2


In [0]:
final_result.to_csv('submission.csv', index = False)